In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import pandas as pd
import psycopg2 as pg

In [11]:
df = 'senate_model'
engine = pg.connect("dbname='postgres' user='postgres' host='elections.c8frdoyd7uok.us-east-2.rds.amazonaws.com' port='5432' password='Flowers89!'")
df = pd.read_sql(f'select * from {df}', con=engine)
df

,index,Unnamed: 0,year,_state,state_code,state_fips,state_cen,state_ic,office,district,...,candidate,party_detailed,writein,_mode,candidatevotes,totalvotes,unofficial,_version,party_simplified,Results
0,0,0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,...,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN,0
1,1,1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,...,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER,0
2,2,2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,...,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT,1
3,3,3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,...,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN,0
4,4,4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,...,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,3624,3009,2014,SOUTH DAKOTA,SD,46,45,37,US SENATE,statewide,...,RICK WEILAND,DEMOCRAT,False,total,82456,279412,False,20210114,DEMOCRAT,0
3625,3625,3248,2016,SOUTH DAKOTA,SD,46,45,37,US SENATE,statewide,...,JOHN R. THUNE,REPUBLICAN,False,total,265516,369656,False,20210114,REPUBLICAN,1
3626,3626,3249,2016,SOUTH DAKOTA,SD,46,45,37,US SENATE,statewide,...,JAY WILLIAMS,DEMOCRAT,False,total,104140,369656,False,20210114,DEMOCRAT,0
3627,3627,3595,2020,SOUTH DAKOTA,SD,46,45,37,US SENATE,statewide,...,MIKE ROUNDS,REPUBLICAN,False,total,276232,420219,True,20210114,REPUBLICAN,1


In [12]:
# Create category_columns and numeric_columns variables
numeric_columns = []
category_columns = []
for col in df.columns:
    if is_string_dtype(df[col]) == True:
        category_columns.append(col)
    elif is_numeric_dtype(df[col]) == True:
        numeric_columns.append(col)
print(numeric_columns)
print(category_columns)

['index', 'Unnamed: 0', 'year', 'state_fips', 'state_cen', 'state_ic', 'special', 'writein', 'candidatevotes', 'totalvotes', 'unofficial', '_version', 'Results']
['_state', 'state_code', 'office', 'district', 'stage', 'candidate', 'party_detailed', '_mode', 'party_simplified']


In [13]:
 #Create dummy variables for the category_columns and merge on the numeric_columns to create an X dataset
category_columns = pd.get_dummies(df[category_columns])
category_columns
X = df[numeric_columns].merge(category_columns, left_index= True, right_index= True)
X

,index,Unnamed: 0,year,state_fips,state_cen,state_ic,special,writein,candidatevotes,totalvotes,...,party_detailed_WORKERS,party_detailed_WORKERS AGAINST CONCESSIONS,party_detailed_WORKERS LEAGUE,party_detailed_WORKERS WORLD,party_detailed_WORKING FAMILIES,_mode_total,party_simplified_DEMOCRAT,party_simplified_LIBERTARIAN,party_simplified_OTHER,party_simplified_REPUBLICAN
0,0,0,1976,4,86,61,False,False,321236,741210,...,0,0,0,0,0,1,0,0,0,1
1,1,1,1976,4,86,61,False,False,1565,741210,...,0,0,0,0,0,1,0,0,1,0
2,2,2,1976,4,86,61,False,False,400334,741210,...,0,0,0,0,0,1,1,0,0,0
3,3,3,1976,4,86,61,False,False,7310,741210,...,0,0,0,0,0,1,0,1,0,0
4,4,4,1976,4,86,61,False,False,10765,741210,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,3624,3009,2014,46,45,37,False,False,82456,279412,...,0,0,0,0,0,1,1,0,0,0
3625,3625,3248,2016,46,45,37,False,False,265516,369656,...,0,0,0,0,0,1,0,0,0,1
3626,3626,3249,2016,46,45,37,False,False,104140,369656,...,0,0,0,0,0,1,1,0,0,0
3627,3627,3595,2020,46,45,37,False,False,276232,420219,...,0,0,0,0,0,1,0,0,0,1


In [14]:
y = df['totalvotes'].values
y

array([741210, 741210, 741210, ..., 369656, 420219, 420219])

In [15]:
# Split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
# Scale X_train and X_test
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
len(X_train_scaled[0])

2673

In [17]:
# Create a neural network model with keras
nn = tf.keras.models.Sequential()

In [18]:
#ONE HIDDEN LAYER
n_input = len(X_train_scaled[0])
n_hidden = n_input * 2
#n_hidden_layer2 = n_input * 2 #2nd hidden layer

nn.add(tf.keras.layers.Dense(units=n_hidden, input_dim=n_input, activation='relu'))
#nn.add(tf.keras.layers.Dense(units=n_hidden_layer2, activation='relu')) #2nd hidden layer

# add an output layer with a 'linear' activation function.
nn.add(tf.keras.layers.Dense(units=1,  activation='linear'))
# print a summary of the model
print(nn.summary())
# compile the model using the "adam" optimizer and "mean_squared_error" loss function
nn.compile(loss='mean_squared_error' , optimizer='adam' , metrics=['mse'])
# train the model for 100 epochs
model = nn.fit(X_train_scaled, y_train, epochs=100)
# predict values for the train and test sets
y_train_pred = nn.predict(X_train_scaled)
y_test_pred = nn.predict(X_test_scaled)
# score the training predictions with r2_score()
print(f"r2_score of y_train: {r2_score(y_train, y_train_pred)}")
# score the test predictions with r2_score()
print(f"r2_score of y_test: {r2_score(y_test, y_test_pred)}")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5346)              14295204  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5347      
Total params: 14,300,551
Trainable params: 14,300,551
Non-trainable params: 0
_________________________________________________________________
None
Train on 2721 samples
Epoch 1/100
2721/2721 [==============================] - 9s 3ms/sample - loss: 9392873385028.1152 - mse: 9392873275392.0000
Epoch 2/100
2721/2721 [==============================] - 9s 3ms/sample - loss: 9389088473654.0039 - mse: 9389086867456.0000
Epoch 3/100
2721/2721 [==============================] - 9s 3ms/sample - loss: 9380036063865.7422 - mse: 9380035559424.0000
Epoch 4/100
2721/2721 [==============================] - 8s 3ms/sample - loss: 9365595619809.7051 - mse: 

In [19]:
#TWO HIDDEN LAYERS
n_input = len(X_train_scaled[0])
n_hidden = n_input * 2
n_hidden_layer2 = n_input * 2 #2nd hidden layer

nn.add(tf.keras.layers.Dense(units=n_hidden, input_dim=n_input, activation='relu'))
nn.add(tf.keras.layers.Dense(units=n_hidden_layer2, activation='relu')) #2nd hidden layer

# add an output layer with a 'linear' activation function.
nn.add(tf.keras.layers.Dense(units=1,  activation='linear'))
# print a summary of the model
print(nn.summary())
# compile the model using the "adam" optimizer and "mean_squared_error" loss function
nn.compile(loss='mean_squared_error' , optimizer='adam' , metrics=['mse'])
# train the model for 100 epochs
model = nn.fit(X_train_scaled, y_train, epochs=100)
# predict values for the train and test sets
y_train_pred = nn.predict(X_train_scaled)
y_test_pred = nn.predict(X_test_scaled)
# score the training predictions with r2_score()
print(f"r2_score of y_train: {r2_score(y_train, y_train_pred)}")
# score the test predictions with r2_score()
print(f"r2_score of y_test: {r2_score(y_test, y_test_pred)}")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5346)              14295204  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5347      
_________________________________________________________________
dense_2 (Dense)              (None, 5346)              10692     
_________________________________________________________________
dense_3 (Dense)              (None, 5346)              28585062  
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5347      
Total params: 14,300,551
Trainable params: 14,300,551
Non-trainable params: 0
_________________________________________________________________
None
Train on 2721 samples
Epoch 1/100
2721/2721 [==============================] - 29s 11ms/sample - loss: 